In [1]:
import findspark

# spark location on namenode server
findspark.init("/usr/hdp/current/spark2-client")
import pyspark
conf = pyspark.SparkConf().setAll([('spark.app.name', 'guobiao_tsp_tbls.guobiao_vehilce_filter'), # App Name
    ('spark.master', 'yarn'),              # spark run mode: locally or remotely
    ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
    ('spark.executor.memory', '8g'),       # memory allocated for each executor
    ('spark.executor.cores', '3'),         # number of cores for each executor
    ('spark.executor.instances', '1'),    # number of executors in total
    ('spark.driver.maxResultSize', '5g'), # Result size is large, need to increase from default of 1g
    ('spark.yarn.am.memory', '10g')])       # memory for spark driver (application master)
sc = pyspark.SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SparkSession 
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [2]:
from datetime import datetime, timedelta
d = datetime.now().date()
delta = timedelta(days=2)
days=[]
for i in range(delta.days+1):
    days.append(d-timedelta(i))

In [3]:
"""
import sys
   
# Import path to zipfile
ZIP_FILE = '/home/wchen/guobiao/veh.zip'
sys.path.insert(0, ZIP_FILE)
from trip_based_stats.configs import *
import trip_based_stats.single_vin_battery_flow as bf
from common.vehicle_utils import load_header_config

config_file = '/home/wchen/guobiao/guobiao_header.csv'
"""

"\nimport sys\n   \n# Import path to zipfile\nZIP_FILE = '/home/wchen/guobiao/veh.zip'\nsys.path.insert(0, ZIP_FILE)\nfrom trip_based_stats.configs import *\nimport trip_based_stats.single_vin_battery_flow as bf\nfrom common.vehicle_utils import load_header_config\n\nconfig_file = '/home/wchen/guobiao/guobiao_header.csv'\n"

In [4]:
import subprocess

def run_cmd(args_list):
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.communicate()
    return proc.returncode

In [5]:
def parseline(line):
    fields=line.split(',')
    vin=str(fields[0])    #no conversion would be string
    #other_fields=[]
    vintype=str(fields[1])
    veh_insulation=str(fields[13])
    esd_sc_volt_list=str(fields[77]).split('|')  
    #esd_sc_volt_list=list(map(float, str(fields[77]).split('|')))        
    #other_fields.append(vintype)
    #other_fields.append(veh_insulation)
    #other_fields.append(esd_sc_volt_list)
    return (vin, vintype, [esd_sc_volt_list, [veh_insulation]])
    #return (vin, vintype, veh_insulation, esd_sc_volt_list)

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark=SparkSession.builder.config("spark.sql.warehouse.dir").appName("SparkSQL").getOrCreate()

data_files=[]
for day in days:
    day = day.isoformat()
    dd = day.replace('-', '')
    df_name='hdfs://172.15.7.170:8020/data/guobiao/csv/d={}'.format(dd)
    returncode = run_cmd(['hdfs', 'dfs', '-test', '-e', df_name])
    if returncode:
        print('{} does not exist, skipping ..'.format(df_name))
        continue    
    data_files.append(df_name)



if len(data_files) == 1:
    rdd = spark.sparkContext.textFile(data_files[0])
else:
    rdds = []
    for data_file in data_files:

        rddi = spark.sparkContext.textFile(data_file)
        rdds.append(rddi)
        rdd = sc.union(rdds)

In [7]:
ge3Lines=rdd.map(parseline).filter(lambda x: "A5HEV" in x[1])

In [8]:
ge3df=ge3Lines.toDF()
ge3df.createOrReplaceTempView("ge3_df")
maxSql = spark.sql("""
SELECT *
FROM ge3_df
LIMIT 5
""")
maxSql.show()

+-----------------+-----+--------------------+
|               _1|   _2|                  _3|
+-----------------+-----+--------------------+
|LMGHP1S57H1000607|A5HEV|[WrappedArray(3.6...|
|LMGHP1S54J1003938|A5HEV|[WrappedArray(3.9...|
|LMGHP1S54J1002806|A5HEV|[WrappedArray(3.9...|
|LMGHP1S83J1001897|A5HEV|[WrappedArray(4.0...|
|LMGHP1S5XJ1003751|A5HEV|[WrappedArray(3.6...|
+-----------------+-----+--------------------+



In [9]:
#ge3Lines.toDF().take(3)
midresult = ge3Lines.map(lambda x: (x[0], x[2])).reduceByKey(lambda x, y: [list(x[0]) + list(y[0]), list(x[1]) + list(y[1])])
#midresult = ge3Lines.map(lambda x: (x[0], x[2])).reduceByKey(lambda x, y: (list(x) + list(y)))

    #.map(lambda x: (x[0], max(float(x[1])), min(float(x[1])), sum(float(x[1])), len(x[1])))
    #.reduceByKey(lambda x, y: (max(x[0], y[0]), min(x[1], y[1]), sum(x[2], y[2]), sum(x[3], y[3]))).collect()
#    
#volt = midresult.reduceByKey(lambda x, y: (min(x,y),max(x,y),(x+y)))
#df=volt.toDF("vin","min","max","sum")

In [10]:
midresult = midresult.map(lambda x: (x[0], [float(i) for i in x[1][0] if i and i.replace('.', '', 1).isdigit()], [float(i) for i in x[1][1] if i and i.replace('.', '', 1).isdigit()]))\
            .filter(lambda x: x[1] and x[2])\
            .map(lambda x: (x[0], max(x[1]), min(x[1]), sum(x[1]), len(x[1]), max(x[2]), min(x[2]), sum(x[2]), len(x[2])))\
            .map(lambda x: (x[0], x[1], x[2], x[3] / x[4], x[5], x[6], x[7] / x[8]))
#midresult = midresult.map(lambda x: (x[0], [float(i) for i in x[1] if i and i.replace('.', '', 1).isdigit()]))\
#            .map(lambda x: (x[0], max(x[1]), min(x[1]), sum(x[1]), len(x[1])))\
#            .map(lambda x: (x[0], x[1], x[2], x[3] / x[4]))

In [11]:
resultDf=midresult.toDF()

In [12]:
resultDf.take(3)

[Row(_1=u'LMGHP1S57J1003111', _2=4.2, _3=3.497, _4=3.8754369751912128, _5=1000.0, _6=1000.0, _7=1000.0),
 Row(_1=u'LMGHP1S52J1004537', _2=3.688, _3=3.498, _4=3.6552043117745754, _5=1000.0, _6=1000.0, _7=1000.0),
 Row(_1=u'LMGHP1S50H1000951', _2=4.199, _3=3.583, _4=3.9384253291203026, _5=1000.0, _6=1000.0, _7=1000.0)]

In [13]:
import findspark

# spark location on namenode server
findspark.init("/usr/hdp/current/spark2-client")
import pyspark
conf = pyspark.SparkConf().setAll([('spark.app.name', 'guobiao_tsp_tbls.guobiao_vehilce_filter'), # App Name
    ('spark.master', 'yarn'),              # spark run mode: locally or remotely
    ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
    ('spark.executor.memory', '8g'),       # memory allocated for each executor
    ('spark.executor.cores', '3'),         # number of cores for each executor
    ('spark.executor.instances', '1'),    # number of executors in total
    ('spark.driver.maxResultSize', '5g'), # Result size is large, need to increase from default of 1g
    ('spark.yarn.am.memory', '10g')])       # memory for spark driver (application master)
sc = pyspark.SparkContext.getOrCreate(conf=conf)

from pyspark.sql import HiveContext
hc = HiveContext(sc)
resultDf.registerTempTable('result_Df')
sql_cmd = """INSERT OVERWRITE TABLE guobiao_tsp_tbls.guobiao_vehicle_filter
              SELECT * FROM result_Df"""
print(sql_cmd)
hc.sql(sql_cmd).show()
sc.stop()
print("done.")

INSERT OVERWRITE TABLE guobiao_tsp_tbls.guobiao_vehicle_filter
              SELECT * FROM result_Df
++
||
++
++

done.
